In [ ]:
from tkinter import *
window = Tk()

In [1]:
import keyboard
from time import time

lastTime = time()
freq = 50
processTime = 1 / freq

while True:
    currTime = time()
    if currTime > (lastTime+processTime):
        if keyboard.is_pressed('a'):
            print('a')
        if keyboard.is_pressed('b'):
            print('b')
        
        keyboard.add_abbreviation
        if keyboard.is_pressed('q'):
            break
            
        lastTime = currTime

In [38]:
import pygame, sys
import math
import numpy as np

#import helper as hp
def scale_img(img, scale):
    size = round(img.get_width()*scale), round(img.get_height()*scale)
    return pygame.transform.scale(img,size)

def blit_rotate_center(win, image, top_left, angle):
    rotated_image = pygame.transform.rotate(image, angle)
    new_rect = rotated_image.get_rect(
        center=image.get_rect(topleft=top_left).center)
    win.blit(rotated_image, new_rect.topleft)
    
def draw_object_mask(win, image, top_left, angle):
    rotated_image = pygame.transform.rotate(image, angle)
    new_rect = rotated_image.get_rect(
        center=image.get_rect(topleft=top_left).center)
    
    pygame.draw.rect(screen, (200,150,150), new_rect, width=1)
    
def linearInterp(A, B, t):
        return A + ((B - A) * t)



GAME_WIDTH = 800
GAME_HEIGHT = 1000

ROAD_WIDTH = 400
ROAD_COLOR = (119,136,153)

BACKROUND_COLOR = (192,192,192)

PLAYER_CAR = scale_img(pygame.image.load('carIMG_updated.png'),0.4)


SCREEN_UPDATE = pygame.USEREVENT

FPS = 30


class Car():
    
    
    def __init__(self, x, y, width, height):
        self.image = PLAYER_CAR
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        
        self.color = (173, 216, 230)
        
        #self.canvas = canvas
        
        self.velocity = 0
        self.acceleration = 0.4
        self.maxVelo = 5
        self.friction = 0.1
        
        self.angle = 0
        self.rotSpeed = 1
        
        self.controller = Controller()
        
    def getConditions(self):
        return [self.controller.forward, self.controller.reverse, self.controller.right, self.controller.left]
        
    
    def draw(self, screen, drawMask=0):
        #car_rect = pygame.Rect(self.x-self.width/2,self.y-self.height/2, self.width, self.height)
        #pygame.draw.rect(screen,self.color,car_rect)
        blit_rotate_center(screen, self.image, (self.x-self.image.get_width()/2,self.y-self.image.get_height()/2), self.angle)
        if drawMask == 1:
            draw_object_mask(screen, self.image, (self.x-self.image.get_width()/2,self.y-self.image.get_height()/2), self.angle)
        
        
        
    def update(self):
        self.__move()
        
    
    def __move(self):
        if (self.controller.forward):
            self.velocity += self.acceleration
        if (self.controller.reverse):
            self.velocity -= self.acceleration
        
        if (self.velocity > self.maxVelo):
            self.velocity = self.maxVelo
            
            #divide here
        if (self.velocity < -self.maxVelo/1.75):
            self.velocity = -self.maxVelo/1.75
            
        if (self.velocity > 0 ):
            self.velocity -= self.friction
        
        if (self.velocity < 0 ):
            self.velocity += self.friction
        
        if (abs(self.velocity) <= self.friction):
            self.velocity = 0
            
        if (self.velocity != 0):
            flip = 1 if self.velocity>0 else -1
        
            if (self.controller.right):
                self.angle -= self.rotSpeed*flip
            if (self.controller.left):
                self.angle += self.rotSpeed*flip
        
        #self.y -= self.velocity
        
        angle_rad = math.radians(self.angle)
        vert = math.cos(angle_rad) * self.velocity
        horiz = math.sin(angle_rad) * self.velocity
        
        self.y -= vert
        self.x -= horiz
        
    
    def updateFriction(self, newFriction):
        self.friction = newFriction
        
    def collide(self, mask, surface, x=0, y=0):
        #x=self.image.get_width()
        #y=self.image.get_height()
        car_mask = pygame.mask.from_surface(self.image)
        #olist = car_mask.outline()
        #pygame.draw.lines(surface, (200,150,150),1,olist)
        offset = (int(self.x-x), int(self.y-y))
        intersection = mask.overlap(car_mask, offset)
        return intersection
        
    def bounce(self):
        self.velocity = -self.velocity
        
        

class Controller():
    
    def __init__(self):
        self.forward = False
        self.reverse = False
        self.right = False
        self.left = False


class Road():
    
    def __init__(self, x, width, numberLanes = 3, color = (119,136,153)):
        self.x = x
        self.width = width
        self.numberLanes = numberLanes
        self.color = color
        
        self.leftShoulder = x - width/2
        self.rightShoulder = x + width/2
        
        length = 999999
        self.top = -length
        self.bottom = length
        
        self.outLineWidth = 5
        self.outLineColor = (255,255,255)
        
        self.laneWidth = 5
        self.laneColor = (139,156,173)
        
        
    def draw(self, screen, drawMask = 0):
        #road_outline = pygame.Rect(self.leftShoulder-self.outLineWidth,self.top,self.width+2*self.outLineWidth, self.bottom-self.top)
        #pygame.draw.rect(screen, self.outLineColor, road_outline)
        
        #Draw the Road
        road_rect = pygame.Rect(self.leftShoulder,self.top, self.width, self.bottom-self.top)
        pygame.draw.rect(screen, self.color, road_rect)
        
        #Draw the lines between Lanes
        for i in range(self.numberLanes):
            x = linearInterp(self.leftShoulder, self.rightShoulder, i/self.numberLanes)
            #road_lane = pygame.Rect(x-self.laneWidth/2, self.top, self.laneWidth, self.bottom-self.top)
            pygame.draw.line(screen, self.laneColor, (x,self.top), (x,self.bottom), self.laneWidth)
            
        #Draw the outside Bariers
        pygame.draw.line(screen, self.outLineColor, (self.leftShoulder,self.top),(self.leftShoulder,self.bottom), self.outLineWidth)
        pygame.draw.line(screen, self.outLineColor, (self.rightShoulder,self.top),(self.rightShoulder,self.bottom), self.outLineWidth)
        
        
        if drawMask == 1:
            self.drawRoadMask(screen)
        
        
        
    def getLaneCenter(self, laneIndex):
        laneWidth = self.width / self.numberLanes
        return self.leftShoulder + laneWidth/2 + laneIndex*laneWidth
    
    def outsideBorderMask(self):
        border_mask = pygame.Surface((GAME_WIDTH, GAME_HEIGHT), pygame.SRCALPHA)
        pygame.draw.line(border_mask, self.outLineColor, (self.leftShoulder/2,self.top),(self.leftShoulder/2,self.bottom), int(self.leftShoulder))
        pygame.draw.line(border_mask, self.outLineColor, (self.rightShoulder + self.leftShoulder/2,self.top),(self.rightShoulder + self.leftShoulder/2,self.bottom), int(self.leftShoulder))
        mask = pygame.mask.from_surface(border_mask)
        return mask
    
    def drawRoadMask(self,screen):
        pygame.draw.line(screen, (200,150,150), (self.leftShoulder/2,self.top),(self.leftShoulder/2,self.bottom), int(self.leftShoulder))
        pygame.draw.line(screen, (200,150,150), (self.rightShoulder + self.leftShoulder/2,self.top),(self.rightShoulder + self.leftShoulder/2,self.bottom), int(self.leftShoulder))
        
    
pygame.init()
screen = pygame.display.set_mode((GAME_WIDTH,GAME_HEIGHT))
clock = pygame.time.Clock()
#road = pygame.Surface((ROAD_WIDTH,GAME_HEIGHT))
#road.fill(pygame.Color(ROAD_COLOR))

playerRoad = Road(GAME_WIDTH/2, ROAD_WIDTH)


#playerCar = Car(playerRoad.getLaneCenter(1)-70,500,50,70)
playerCar = Car(playerRoad.getLaneCenter(0),500,50,100)


#Add timer for player input in ms
pygame.time.set_timer(SCREEN_UPDATE, 20)


while True:
    #draw all elements
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            #sys.exit()
        if event.type == SCREEN_UPDATE:
            playerCar.update()
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                playerCar.controller.forward = True
            if event.key == pygame.K_DOWN:
                playerCar.controller.reverse = True
            if event.key == pygame.K_RIGHT:
                playerCar.controller.right = True
            if event.key == pygame.K_LEFT:
                playerCar.controller.left = True
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_UP:
                playerCar.controller.forward = False
            if event.key == pygame.K_DOWN:
                playerCar.controller.reverse = False
            if event.key == pygame.K_RIGHT:
                playerCar.controller.right = False
            if event.key == pygame.K_LEFT:
                playerCar.controller.left = False
                
        if playerCar.collide(playerRoad.outsideBorderMask(), screen, x=playerCar.image.get_width()/2) != None:
            #print('collide')
            playerCar.bounce()
            
    screen.fill(pygame.Color(BACKROUND_COLOR))
    #screen.blit(road,(GAME_WIDTH/2 - ROAD_WIDTH/2,0))
    
    #screen.blit(playerRoad.outsideBorderMask().to_surface(), (0,0))
    
    playerRoad.draw(screen,1)
    #playerRoad.drawRoadMask(screen)
    playerCar.draw(screen,1)
    
    
    pygame.display.update()
    clock.tick(FPS)

error: display Surface quit